In [ ]:
#workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
import os
This_module_path = os.getcwd()  #os.path.dirname(os.path.abspath(__file__))
workdir_ = os.path.join(This_module_path, '../../' )
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")

print( f" In {__name__} we have This_module_path={This_module_path} " )
print( f" In {__name__} we have workdir_={workdir_} " )

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri


# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime



########################################
# Now import your own stuff
########################################

#From in here
import gw_common as GWc
import gw_movmtn as GWmm

sys.path.append(workdir_ + 'myPythonTools/Utils/')
import AveragingUtils as Av
import PlotUtil as Pu
import utils as uti
import numerical_utils as nuti
import validation_data as Val
import shr_const
from shr_const import ShrConst as Cs

sys.path.append(workdir_ + 'myPythonTools/Plotting/')
import LatLonMaps as LL


sys.path.append(workdir_ + 'PyRegridding/Regridder/')
import var_A_x_B as vAB

sys.path.append(workdir_ + 'PyRegridding/Utils/')
import MakePressures as MkP
import GridUtils as GrU



importlib.reload( GWmm )
importlib.reload( GWc )
importlib.reload( LL )
importlib.reload( uti )
importlib.reload( nuti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:

timetag = '2010-12-20-*'
expA = "c160mm_ne30pg3_FMTndg_cciasi_x05"
A1 = uti.MakeDict4Exp( exp=expA ,
                     user='juliob',
                     subd='hist',  #cfg[x_]['grid'] , 
                     hsPat='cam.h1i',
                     ymdPat= timetag ,
                     verbose=True, open_dataset=True )

A2 = uti.MakeDict4Exp( exp=expA ,
                     user='juliob',
                     subd='hist',  #cfg[x_]['grid'] , 
                     hsPat='cam.h2a',
                     ymdPat= timetag ,
                     verbose=True, open_dataset=True )

A3 = uti.MakeDict4Exp( exp=expA ,
                     user='juliob',
                     subd='hist',  #cfg[x_]['grid'] , 
                     hsPat='cam.h3i',
                     ymdPat= timetag ,
                     verbose=True, open_dataset=True )

A1v = uti.MakeDict4Exp( exp=expA ,
                     user='juliob',
                     subd='hist',  #cfg[x_]['grid'] , 
                     hsPat='cam.h1i.TAU_vort500',
                     ymdPat= timetag ,
                     verbose=True, open_dataset=True )

A1u = uti.MakeDict4Exp( exp=expA ,
                     user='juliob',
                     subd='hist',  #cfg[x_]['grid'] , 
                     hsPat='cam.h1i.TAU_uniform',
                     ymdPat= timetag ,
                     verbose=True, open_dataset=True )


In [ ]:
A3.X

In [ ]:

#A1.X
SourceMethod="vort500"
nt,nz,ncol = np.shape( A1.X.UEGW.values )
zetaX = np.zeros( (nt,ncol) )
if (SourceMethod=="vort500"):
    k5000=68 # ~500hPa surface in L93
    for t in np.arange( nt ):
        u_se = A1.X.UEGW[t,k5000,:].values
        v_se = A1.X.VEGW[t,k5000,:].values        
        u_fv,lat_fv,lon_fv = vAB.Hregrid( avar=u_se, agrid='ne30pg3', bgrid='fv1x1' )
        v_fv,lat_fv,lon_fv = vAB.Hregrid( avar=v_se, agrid='ne30pg3', bgrid='fv1x1' )
        zeta_fv = nuti.Sphere_Curl2( u_fv , v_fv , lat_fv, lon_fv, wrap=True )
        zeta_se,lat_se,lon_se = vAB.Hregrid( avar=zeta_fv , agrid='fv1x1' ,bgrid='ne30pg3')
        zetaX[t,:] = zeta_se #np.expand_dims( zeta_se, axis=0 )
        print(f"zeta for {t}")
    Dar = xr.DataArray( data=zetaX, 
                    dims=('time','ncol',),
                    attrs=dict( description='Vorticity near 500hPa',units='Pa',) ,) 
    A1.X['ZETA500'] = Dar
 


In [ ]:
A3.X['UVPWP_CLUBB'] = np.sqrt( A3.X['VPWP_CLUBB']**2 + A3.X['UPWP_CLUBB']**2 )

In [ ]:
"""
#plt.contourf( CCor.longitude, CCor.latitude, np.average( amf*nobo,axis=0),levels = np.linspace(0.,.5,num=21) )

Xplo=A1v.X
fplo='TAU_redo'
itv = 3
Xplo=A1.X
fplo='TAU_MOVMTN'
#fields=[  (amf*nobo)[199,:,:]  , 1000.*( nobo*mmfu18)[199,:,:]  ] 
fields=[  1000.*( Xplo[fplo][itv,90,:].values), 1000.*( Xplo[fplo][itv,80,:].values), 1000.*( Xplo[fplo][itv,70,:].values) ,1000.*( Xplo[fplo][itv,60,:].values) ,1000.*( Xplo[fplo][itv,40,:].values)  ] 
topo = A3.X.PHIS.values[0,:]/9.8


clevs= [np.linspace(0.,10,num=21) , np.linspace(0.,10,num=21) , np.linspace(0.,10,num=21) , np.linspace(0.,30,num=31) ]
titles=['CONCORDIASI abs mom flux (AMF) at 18km', '500hPa Vorticity source AMF, 18km ', 'PBL CLUBB mom-flux source AMF, 18km', 'CONCORDIASI # Obs']
LL.Maps_Unstruc (  fields=fields,lats=[Xplo.lat],lons=[Xplo.lon],clevs= clevs ,titles=titles, topo=topo )
"""



In [ ]:
#plt.contourf( CCor.longitude, CCor.latitude, np.average( amf*nobo,axis=0),levels = np.linspace(0.,.5,num=21) )

Xplo=A3.X
fplo='WPTHLP_CLUBB'
fplo='UVPWP_CLUBB'
itv = 0
#fields=[  (amf*nobo)[199,:,:]  , 1000.*( nobo*mmfu18)[199,:,:]  ] 
fields=[]
titles=[]
for kk in np.arange( start=93, stop=78, step=-2):
#for kk in np.arange( start=93, stop=50, step=-5):
    fields.append( Xplo[fplo][itv,kk,:].values ) 
    titles.append(f'{fplo} {A3.X.ilev[kk].values:.0f} hPa')

                         
topo = A3.X.PHIS.values[0,:]/9.8

cmap=['bwr']
cmap=['magma']
clevs= [0.01 * np.linspace(-20.,20,num=21) ]
clevs= [np.linspace(0,0.5,num=21) ]
LL.Maps_Unstruc (  fields=fields,lats=[Xplo.lat],lons=[Xplo.lon],clevs= clevs ,titles=titles, topo=topo,cmaps=cmap )


In [ ]:
wpthlp = A3.X.WPTHLP_CLUBB[0,:,:].values.T
uvpwp = A3.X.UVPWP_CLUBB[0,:,:].values.T
wp2 = A3.X.WP2_CLUBB[0,:,:].values.T
thlp2 = A3.X.THLP2_CLUBB[0,:,:].values.T
zi = A1.X.ZEGW[0,:,:].values.T
lat=A1.X.lat.values
lon=A1.X.lon.values
print( np.shape( wpthlp) )
ncol,pverp = np.shape( wpthlp) 
pver=pverp-1

In [ ]:
pbl_top_u, pbl_top_s, pbl_top_xp2 = GWmm.gw_pbl_top(ncol , pver, wpthlp, uvpwp, wp2, thlp2, zi )

In [ ]:

levels=np.linspace(0,2000,num=21)
shlev=np.linspace( -100,100,num=21)
ztlev=np.linspace( -0.00025,0.00025,num=21)

zeta500 = A1.X.ZETA500[0,:].values
titles = ['PBLTOP unstable', 'PBLTOP stable', 'PBLTOP XP2' ,'ZETA500', 'SHFLX']
LL.Maps_Unstruc (  fields=[pbl_top_u , pbl_top_s , pbl_top_xp2, zeta500,wpthlp[:,93]],lats=[lat],lons=[lon],clevs=[levels,levels,3*levels, ztlev,shlev],topo=topo ,cmaps =['magma','magma','magma','bwr','bwr'],titles=titles) 
#titles = ['PBLTOP unstable', 'PBLTOP unstable .2', 'ZETA500', 'SHFLX']
#LL.Maps_Unstruc (  fields=[pbl_top3,pbl_top_unst,zeta500,wpthlp[:,93]],lats=[lat],lons=[lon],clevs=[levels,levels,ztlev,shlev],topo=topo ,cmaps =['magma','magma','bwr','bwr'],titles=titles) 

In [ ]:
#plt.contourf( CCor.longitude, CCor.latitude, np.average( amf*nobo,axis=0),levels = np.linspace(0.,.5,num=21) )

topo = A3.X.PHIS.values[0,:]/9.8
lons,lats=Xplo.lon,Xplo.lat

In [ ]:
zoo=np.where( (lons>180.)&(lons<220)&(lats>-50.)&(lats<-40.))

In [ ]:
zoo2=np.where( (lons>195.)&(lons<200.)&(lats>-48.)&(lats<-45.)&(A3.X.WPTHLP_CLUBB[0,93,:].values>0.))
zoo3=np.where( (lons>90.)&(lons<110.)&(lats>-55.)&(lats<-45.)&(A3.X.WPTHLP_CLUBB[0,93,:].values<0.)&(pbl_top_s>1000.))

In [ ]:
# Let's make composites .
zoo_u=np.where( (lats<-45.)&(A3.X.WPTHLP_CLUBB[0,93,:].values>0.) &(pbl_top_u>800.)&(pbl_top_u<1000.) )
zoo_s=np.where( (lats<-45.)&(A3.X.WPTHLP_CLUBB[0,93,:].values<0.) &(pbl_top_s>800.)&(pbl_top_s<1000.) ) 
zoo_uu=np.where( (lats<-45.)&(A3.X.WPTHLP_CLUBB[0,93,:].values>20.) ) #&(pbl_top_u>800.)&(pbl_top_u<1000.) )
zoo_ss=np.where( (lats<-45.)&(A3.X.WPTHLP_CLUBB[0,93,:].values<-20.) ) # &(pbl_top_s>800.)&(pbl_top_u<1000.) ) 
zoo_weird = np.where( pbl_top_s>pbl_top_xp2 )

In [ ]:
print( np.shape(zoo_weird[0]) )

In [ ]:

AX=A1.X

pmid = 0.5*( AX.PINT.values[:,:-1,:]+AX.PINT.values[:,1:,:] )
print(np.shape(pmid))
print(np.shape(AX.PINT[:,1:,:]) )
print(np.shape(AX.PINT[:,:-1,:]) )
print(np.shape( AX.PINT[:,1:,:]+AX.PINT[:,:-1,:]) )


In [ ]:
theta = AX.TEGW.values * ( ( 100_000. / pmid )**(2./7) )

In [ ]:
pl_lev=np.average( A1.X.ZMGW.values[ itv,:,zoo_u[0][:] ] ,axis=0 )
pl_lev.shape

In [ ]:
print( theta.shape)

In [ ]:
itv=0
fig,ax=plt.subplots(2 , 3, figsize=(21,14) )
ax=ax.flatten()
n=0
icol = zoo3[0][10]
#icol = 11035
icol = zoo2[0][0] 

icol = zoo_uu[0][312]
#icol = zoo_uu[0][112]

#icol = zoo_weird[0][0]

print(A1.X.lon[icol].values, A1.X.lat[icol].values)
print(f'PRECC={1000.*86_4000.*A3.X.PRECC[itv,icol].values} mm d-1')
print(f'PRECL={1000.*86_4000.*A3.X.PRECL[itv,icol].values} mm d-1')
#, A1.X.lat[icol].values)

print(f'Unstable  PBL top diag {pbl_top_u[icol]}' )
print(f'Stable    PBL top diag {pbl_top_s[icol]}' )
print(f'XP2-based PBL top diag {pbl_top_xp2[icol]}' )
#print(pbl_top_s[icol] )


pl_lev=A1.X.ZMGW[itv,:,icol].values
pl_ilev=A1.X.ZEGW[itv,:,icol].values

ylims=[0,5000.]

ax[n].plot( A1.X.UEGW[itv,:,icol] , pl_lev )
ax[n].plot( A1.X.VEGW[itv,:,icol] , pl_lev )
ax[n].plot( A1.X.UBM_MOVMTN[itv,:,icol] , pl_lev ,'o-' )
ax[n].set_ylim( ylims )
ax[n].set_xlim(-20,40)
ax[n].set_title( "Met Profiles" )


n=n+1
ax[n].plot(theta[itv,:,icol] , pl_lev )
#ax[n].plot( A1.X.VEGW[itv,:,icol] , pl_lev )
#ax[n].plot( A1.X.UBM_MOVMTN[itv,:,icol] , pl_lev ,'o-' )
ax[n].set_ylim( ylims )
ax[n].set_xlim(270,400)
ax[n].set_title( "Theta" )



n=n+1
ax[n].plot( A3.X.UPWP_CLUBB[itv,:,icol] , pl_ilev )
ax[n].plot( A3.X.VPWP_CLUBB[itv,:,icol] , pl_ilev )
ax[n].plot( A3.X.UVPWP_CLUBB[itv,:,icol] , pl_ilev, 'o-' )
#ax[n].plot( A1.X.VEGW[itv,:,icol] , pl_lev )
#ax[n].plot( A1.X.UBM_MOVMTN[itv,:,icol] , pl_lev ,'o-' )
ax[n].set_ylim( ylims )
ax[n].set_xlim(-.1,.5)
ax[n].set_title( "Mom Fluxes" )

n=n+1
ax[n].plot( A3.X.WP2_CLUBB[itv,:,icol] , pl_ilev )
#ax[n].plot( A1.X.VEGW[itv,:,icol] , pl_lev )
#ax[n].plot( A1.X.UBM_MOVMTN[itv,:,icol] , pl_lev ,'o-' )
ax[n].set_ylim( ylims )
#ax[n].set_xlim(-10,20)
ax[n].set_title( "WP2" )

n=n+1
ax[n].plot( A3.X.THLP2_CLUBB[itv,:,icol] , pl_ilev )
#ax[n].plot( A1.X.VEGW[itv,:,icol] , pl_lev )
#ax[n].plot( A1.X.UBM_MOVMTN[itv,:,icol] , pl_lev ,'o-' )
ax[n].set_ylim( ylims )
#ax[n].set_xlim(-10,20)
ax[n].set_title( "THLP2" )

n=n+1
ax[n].plot( A3.X.WPTHLP_CLUBB[itv,:,icol] , pl_ilev , 'o-' )
#ax[n].plot( A1.X.VEGW[itv,:,icol] , pl_lev )
#ax[n].plot( A1.X.UBM_MOVMTN[itv,:,icol] , pl_lev ,'o-' )
ax[n].set_ylim( ylims )
#ax[n].set_xlim(-10,20)
ax[n].set_title( "WPTHLP" )




In [ ]:
print( A3.X.THLP2_CLUBB[itv,:,icol].values )
print( A3.X.WP2_CLUBB[itv,:,icol].values )


In [ ]:

fig,ax=plt.subplots(2 , 3, figsize=(21,14) )
ax=ax.flatten()
itv=0
n=0

zoo_x = zoo_s
pl_lev=np.average( A1.X.ZMGW.values[ itv,:,zoo_x[0][:] ] ,axis=0 )
pl_ilev=np.average( A1.X.ZEGW.values[ itv,:,zoo_x[0][:] ] ,axis=0 )

uegw=np.average( A1.X.UEGW.values[ itv,:,zoo_x[0][:] ] ,axis=0 )
thetaa=np.average( theta[ itv,:,zoo_x[0][:] ] ,axis=0 )

uvpwp=np.average( A3.X.UVPWP_CLUBB.values[ itv,:,zoo_x[0][:] ] ,axis=0 )
wp2=np.average( A3.X.WP2_CLUBB.values[ itv,:,zoo_x[0][:] ] ,axis=0 )
thlp2=np.average( A3.X.THLP2_CLUBB.values[ itv,:,zoo_x[0][:] ] ,axis=0 )
wpthlp=np.average( A3.X.WPTHLP_CLUBB.values[ itv,:,zoo_x[0][:] ] ,axis=0 )

                  
ylims=[0,3000.]

ax[n].plot( uegw , pl_lev  ,color='cyan' )
#ax[n].plot( A1.X.VEGW[itv,:,icol] , pl_lev )
#ax[n].plot( A1.X.UBM_MOVMTN[itv,:,icol] , pl_lev ,'o-' )
ax[n].set_ylim( ylims )
ax[n].set_xlim(-20,40)
ax[n].set_title( "Met Profiles" )


n=n+1
ax[n].plot( thetaa , pl_lev  ,color='cyan' )
#ax[n].plot( A1.X.VEGW[itv,:,icol] , pl_lev )
#ax[n].plot( A1.X.UBM_MOVMTN[itv,:,icol] , pl_lev ,'o-' )
ax[n].set_ylim( ylims )
ax[n].set_xlim(270,400)
ax[n].set_title( "Theta" )



n=n+1
#ax[n].plot( A3.X.UPWP_CLUBB[itv,:,icol] , pl_ilev )
#ax[n].plot( A3.X.VPWP_CLUBB[itv,:,icol] , pl_ilev )
ax[n].plot( uvpwp , pl_ilev  ,color='cyan' )#, 'o-' )
#ax[n].plot( A1.X.VEGW[itv,:,icol] , pl_lev )
#ax[n].plot( A1.X.UBM_MOVMTN[itv,:,icol] , pl_lev ,'o-' )
ax[n].set_ylim( ylims )
ax[n].set_xlim(-.1,.5)
ax[n].set_title( "Mom Fluxes" )


n=n+1
ax[n].plot( wp2 , pl_ilev ,color='cyan'  )
#ax[n].plot( A1.X.VEGW[itv,:,icol] , pl_lev )
#ax[n].plot( A1.X.UBM_MOVMTN[itv,:,icol] , pl_lev ,'o-' )
ax[n].set_ylim( ylims )
#ax[n].set_xlim(-10,20)
ax[n].set_title( "WP2" )



n=n+1
ax[n].plot( thlp2 , pl_ilev ,color='cyan'  )
#ax[n].plot( A1.X.VEGW[itv,:,icol] , pl_lev )
#ax[n].plot( A1.X.UBM_MOVMTN[itv,:,icol] , pl_lev ,'o-' )
ax[n].set_ylim( ylims )
ax[n].set_xlim(0. , .2)
ax[n].set_title( "THLP2" )

n=n+1
ax[n].plot( wpthlp , pl_ilev ,color='cyan' )
#ax[n].plot( A1.X.VEGW[itv,:,icol] , pl_lev )
#ax[n].plot( A1.X.UBM_MOVMTN[itv,:,icol] , pl_lev ,'o-' )
ax[n].set_ylim( ylims )
ax[n].set_xlim(-20,40)
ax[n].set_title( "WPTHLP" )




In [ ]:
#plt.scatter( A1.X.UEGW[0,:,zoo2[0][0]]  , Xplo.lev )
plt.plot( A1.X.UEGW[0,70:80,zoo2[0][0]]  ,A1.X.VEGW[0,70:80,zoo2[0][0]] ,'o-')
plt.ylim( -5.,5.)
plt.xlim(-5,5)

In [ ]:
A1.X

In [ ]:
import numpy as np
import xarray as xr

# Create an example DataArray with shape (4, 94, 48600)
data = np.random.rand(4, 94, 48600)
AA = xr.Dataset({'PINT': (('time', 'level', 'point'), data)})

# Slicing the DataArray
slice_1 = AA.PINT[:, 1:, :].shape  # (4, 93, 48600)
slice_2 = AA.PINT[:, :-1, :].shape  # (4, 93, 48600)

# Adding the slices results in unexpected shape reduction
result = AA.PINT[:, 1:, :] + AA.PINT[:, :-1, :]
result_shape = result.shape  # (4, 92, 48600)

print("Shape of original PINT:", AA.PINT.shape)  # (4, 94, 48600)
print("Shape after slicing 1:", slice_1)  # (4, 93, 48600)
print("Shape after slicing 2:", slice_2)  # (4, 93, 48600)
print("Shape after addition:", result_shape)  # Expected (4, 93, 48600), but got (4, 92, 48600)
